# Dataset labeller
---

This file contains a widget to assist in the labelling of the data.

In [1]:
import ipywidgets as widgets
from pathlib import Path
from shutil import copy
from functools import partial
from os import remove, walk
from IPython.display import clear_output

In [6]:
DATASET = Path('Models', 'Dataset', 'Wall Recognizer', 'Raw')
OUTPUT = Path('Models', 'Dataset', 'Wall Recognizer', 'Labelled')

def save_template(file_name:str, top_border: bool, bottom_border: bool, 
                  left_border: bool, right_border: bool):
    input = DATASET/file_name
    output = OUTPUT/f"{1 if top_border else 0}_{1 if bottom_border else 0}_{1 if left_border else 0}_{1 if right_border else 0}_{file_name}"
    copy(input, output)
    remove(input)

def on_click(context, file_names:list[str], file_name:str, grid):
    values = [file_name]
    for i in range(4):
        values.append(grid[i, 1].value)
    save_template(*values)
    clear_output()
    label(file_names)

def label(file_names: list[str]):
    if len(file_names) == 0:
        return
    file_name = file_names.pop()
    file_path = DATASET/file_name
    image = open(file_path, 'rb').read()
    grid = widgets.GridspecLayout(6, 2)
    grid[:, 0] = widgets.Image(
        value=image,
        format='png',
        width=300,
        height=400,
        layout=widgets.Layout(margin='0', padding='0', border='0')
    )

    checkboxes = ['Top Border', 'Bottom Border', 'Left Border', 'Right Border']
    for i, x in enumerate(checkboxes):
        grid[i, 1] = widgets.Checkbox(
            value=False,
            description=x,
            dosabled=False,
            indent=False
        )
    grid[4, 1] = widgets.Button(
        description='Save',
        disabled=False,
        button_style='success',
        icon='check',
        layout=widgets.Layout(width='70%')
    )
    grid[4, 1].on_click(partial(on_click, file_names=file_names, file_name=file_name, grid=grid))
    grid[5, 1] = widgets.Button(
        description='Stop',
        disabled=False,
        button_style='warning',
        icon='cross',
        layout=widgets.Layout(width='70%')
    )
    grid[5, 1].on_click(lambda x: exit())
    display(grid)
    return grid[5, 1]

In [ ]:
images = []

for root, dirs, files in walk(DATASET, topdown=False):
   for name in files:
      if '.png' in name:
         images.append(name)

_ = label(images)